<a href="https://colab.research.google.com/github/Jonathan-C-Barrett/GEOG5990M/blob/main/GEOG5003M_Final_Project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5003M Final Assignment (Template)

Student ID number:

GitHub repo link:

Word count limit= 1,500 words max (markdown cells only, excluding readme)

In [1]:
# read in required packages

import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

import matplotlib.pyplot as plt

!pip install mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 24.0 MB/s eta 0:00:00


In [ ]:
#Read in Health Services Data
ahah = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/ahah_v4.csv')
ahah.head()

In [ ]:
#Select only columns relating to Health Services
health = ahah[['LSOA21CD', 'ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h']]
health.head()

In [ ]:
#check for missing values
health.isna().sum()

In [ ]:
#Read in Median Property Sales
m_sales = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/land-registry-house-prices-LSOA.csv')
m_sales.head()

In [ ]:
#Select only columns relating to 2016
prop_16 = m_sales[['Code', 'Area','Year ending Mar 2016','Year ending Jun 2016', 'Year ending Sep 2016', 'Year ending Dec 2016']]
prop_16.head()

In [ ]:
#check for missing values
prop_16.isna().sum()

In [ ]:
#remove NA values
prop_16 = prop_16.dropna()
prop_16.isna().sum()

In [ ]:
#examine property info for count and data type
prop_16.info()

In [ ]:
#add conversion to LSOA21
LSOA21 = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Best_Fit_Lookup_for_EW_(V2).csv')
LSOA21.head()


In [ ]:
#inspect info for data type
LSOA21.info()

In [16]:
#combine prop_16 data and LSOA21 by code in prop_16 and LSOA11CD in LSOA21 in to remove areas outside london and make comparable
prop_16_LSOA21 = pd.merge(prop_16, LSOA21, left_on='Code', right_on='LSOA11CD', how='left')
prop_16_LSOA21.head()

,Code,Area,Year ending Mar 2016,Year ending Jun 2016,Year ending Sep 2016,Year ending Dec 2016,ObjectId,LSOA11CD,LSOA11NM,LSOA21CD,LSOA21NM,LAD22CD,LAD22NM,LAD22NMW
0,E01000001,City of London 001A,910000,975000,999000,985000,1,E01000001,City of London 001A,E01000001,City of London 001A,E09000001,City of London,NaN
1,E01000002,City of London 001B,880000,845000,842500,845000,2,E01000002,City of London 001B,E01000002,City of London 001B,E09000001,City of London,NaN
2,E01000003,City of London 001C,630000,685000,745000,717500,3,E01000003,City of London 001C,E01000003,City of London 001C,E09000001,City of London,NaN
3,E01000005,City of London 001E,570500,585000,556000,556000,4,E01000005,City of London 001E,E01000005,City of London 001E,E09000001,City of London,NaN
4,E01032739,City of London 001F,599975,605500,582500,585000,31675,E01032739,City of London 001F,E01032739,City of London 001F,E09000001,City of London,NaN


In [17]:
#check Info for count
prop_16_LSOA21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4835 entries, 0 to 4834
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Code                  4835 non-null   object
 1   Area                  4835 non-null   object
 2   Year ending Mar 2016  4835 non-null   object
 3   Year ending Jun 2016  4835 non-null   object
 4   Year ending Sep 2016  4835 non-null   object
 5   Year ending Dec 2016  4835 non-null   object
 6   ObjectId              4835 non-null   int64 
 7   LSOA11CD              4835 non-null   object
 8   LSOA11NM              4835 non-null   object
 9   LSOA21CD              4835 non-null   object
 10  LSOA21NM              4835 non-null   object
 11  LAD22CD               4835 non-null   object
 12  LAD22NM               4835 non-null   object
 13  LAD22NMW              0 non-null      object
dtypes: int64(1), object(13)
memory usage: 529.0+ KB


In [22]:
#combine health and property data by LSOA code in property to remove areas outside london
health_prop = pd.merge(health, prop_16_LSOA21, left_on='LSOA21CD', right_on='LSOA21CD', how='right')
health_prop.head()

,LSOA21CD,ah4dent,ah4gp,ah4hosp,ah4phar,ah4leis,ah4h,Code,Area,Year ending Mar 2016,Year ending Jun 2016,Year ending Sep 2016,Year ending Dec 2016,ObjectId,LSOA11CD,LSOA11NM,LSOA21NM,LAD22CD,LAD22NM,LAD22NMW
0,E01000001,0.6328,1.0316,0.9581,0.7487,0.3596,-1.7834,E01000001,City of London 001A,910000,975000,999000,985000,1,E01000001,City of London 001A,City of London 001A,E09000001,City of London,NaN
1,E01000002,0.6230,1.3232,1.1370,0.6849,0.5571,-1.5523,E01000002,City of London 001B,880000,845000,842500,845000,2,E01000002,City of London 001B,City of London 001B,E09000001,City of London,NaN
2,E01000003,0.5709,0.8562,0.7862,0.5919,0.4608,-1.9813,E01000003,City of London 001C,630000,685000,745000,717500,3,E01000003,City of London 001C,City of London 001C,E09000001,City of London,NaN
3,E01000005,0.8337,1.1078,0.8697,0.6657,0.3717,-1.7260,E01000005,City of London 001E,570500,585000,556000,556000,4,E01000005,City of London 001E,City of London 001E,E09000001,City of London,NaN
4,E01032739,0.6641,1.6422,1.1970,0.6810,0.4148,-1.5235,E01032739,City of London 001F,599975,605500,582500,585000,31675,E01032739,City of London 001F,City of London 001F,E09000001,City of London,NaN


In [ ]:
#check info for data frame to check count and data type
health_prop.info()

In [ ]:
#check for missing values
health_prop.isnull().sum()

In [ ]:
#remove duplicated columns from Data base
health_prop_clean = health_prop.drop(columns=['Code', 'Area', 'LSOA11NM', 'LSOA11CD', 'ObjectId','LAD22NMW'])
health_prop_clean.head()


In [49]:
#Some property data missing but replaced with ':' symbol. Replace these with space
health_prop_clean['Year ending Mar 2016']= health_prop_clean['Year ending Mar 2016'].replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)


In [50]:
health_prop_clean['Year ending Jun 2016']= health_prop_clean['Year ending Jun 2016'].replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)

In [52]:
health_prop_clean['Year ending Sep 2016']= health_prop_clean['Year ending Sep 2016'].replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)

In [53]:
health_prop_clean['Year ending Dec 2016']= health_prop_clean['Year ending Dec 2016'].replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)

In [54]:
health_prop_clean.isnull().sum()


,0
LSOA21CD,0
ah4dent,0
ah4gp,0
ah4hosp,0
ah4phar,0
ah4leis,0
ah4h,0
Year ending Mar 2016,91
Year ending Jun 2016,0
Year ending Sep 2016,0


In [56]:
# Change the data type of 'Year ending Mar 2016' to float
health_prop_clean['Year ending Mar 2016'] = health_prop_clean['Year ending Mar 2016'].astype(float)

In [59]:
health_prop_clean['Year ending Jun 2016'] = health_prop_clean['Year ending Jun 2016'].astype(float)

ValueError: could not convert string to float: ' '

In [58]:
health_prop_clean.dtypes

,0
LSOA21CD,object
ah4dent,float64
ah4gp,float64
ah4hosp,float64
ah4phar,float64
ah4leis,float64
ah4h,float64
Year ending Mar 2016,float64
Year ending Jun 2016,object
Year ending Sep 2016,object


# **SPATIAL VISULAISATION **

In [2]:
# Data downloaded from https://geoportal.statistics.gov.uk/datasets/f23c50d2bffa4a8d9693bcf4ed72bdaf_0/explore?location=0.278866%2C-2.487028%2C0.00
shp =gpd.read_file('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/CMLAD_Dec_2011_SGCB_GB_2022_5882430432395265005.geojson')


In [3]:
#inspect data
shp.head()

,OBJECTID,cmlad11cd,cmlad11nm,cmlad11nmw,GlobalID,geometry
0,32,E41000032,Luton,,c0fc3ba7-e2b0-4014-949c-ff4b98184a3e,"POLYGON ((-0.40756 51.91069, -0.40244 51.9097,..."
1,33,E41000033,Southend-on-Sea,,3f2d9e3e-a142-433a-9619-d196f4174c5e,"MULTIPOLYGON (((0.75315 51.55487, 0.76986 51.5..."
2,34,E41000034,Thurrock,,9e42d208-9abf-4e34-b43b-ea7beae6278b,"MULTIPOLYGON (((0.39181 51.56721, 0.39353 51.5..."
3,35,E41000035,Medway,,9bdd1fa6-cbef-4648-af38-af3a1acd741b,"MULTIPOLYGON (((0.55364 51.48494, 0.55456 51.4..."
4,36,E41000036,Bracknell Forest,,69ea3d20-66d6-499e-b7e1-7b5dba88f4a7,"POLYGON ((-0.76463 51.45972, -0.75733 51.45902..."


In [ ]:
#combine boundary data and health and property data through LSOA code
ldn_prop = pd.merge(shp, prop_16, left_on='LSOA21CD', right_on='Code', how='right')
ldn_prop.head()

In [ ]:
#inspect data
ldn_prop.isnull().sum()

In [76]:
ldn_health_prop.to_file('London.geojson')

In [ ]:
ldn_health_prop.explore()

In [81]:
#select only london boroughs
london_shp= shp.loc[shp['LSOA21NM'].str.contains('City of London') |
                    shp['LSOA21NM'].str.contains('Barking and Dagenham')|
                    shp['LSOA21NM'].str.contains('Barnet')|
                    shp['LSOA21NM'].str.contains('Bexley')|
                    shp['LSOA21NM'].str.contains('Brent') |
                    shp['LSOA21NM'].str.contains('Bromley')|
                    shp['LSOA21NM'].str.contains('Camden')|
                    shp['LSOA21NM'].str.contains('Croydon')|
                    shp['LSOA21NM'].str.contains('Ealing')|
                    shp['LSOA21NM'].str.contains('Enfield')|
                    shp['LSOA21NM'].str.contains('Greenwich')|
                    shp['LSOA21NM'].str.contains('Hackney')|
                    shp['LSOA21NM'].str.contains('Hammersmith and Fulham')|
                    shp['LSOA21NM'].str.contains('Haringey')|
                    shp['LSOA21NM'].str.contains('Harrow')|
                    shp['LSOA21NM'].str.contains('Havering')|
                    shp['LSOA21NM'].str.contains('Hillingdon')|
                    shp['LSOA21NM'].str.contains('Hounslow')|
                    shp['LSOA21NM'].str.contains('Islington')|
                    shp['LSOA21NM'].str.contains('Kensington and Chelsea')|
                    shp['LSOA21NM'].str.contains('Kingston upon Thames')|
                    shp['LSOA21NM'].str.contains('Lambeth')|
                    shp['LSOA21NM'].str.contains('Lewisham')|
                    shp['LSOA21NM'].str.contains('Merton')|
                    shp['LSOA21NM'].str.contains('Newham')|
                    shp['LSOA21NM'].str.contains('Redbridge')|
                    shp['LSOA21NM'].str.contains('Richmond upon Thames')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Sutton')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Tower Hamlets')|
                    shp['LSOA21NM'].str.contains('Waltham Forest')|
                    shp['LSOA21NM'].str.contains('Wandsworth')|
                    shp['LSOA21NM'].str.contains('Westminster'),:]

In [84]:
#investigate data
london_shp.isnull().sum()

,0
FID,0
OA21CD,0
LSOA21CD,0
LSOA21NM,0
LSOA21NMW,0
BNG_E,0
BNG_N,0
LAT,0
LONG,0
GlobalID,0


In [ ]:
#explore on leaflet map
london_shp.explore()

In [ ]:
#check for missing values
london_shp.isna().sum()

This is an example block of mardown text I want to reference <a href="#ref1">[1]</a>. I might need to add some more citations <a href="#ref2">[2]</a><a href="#ref2">[3]</a>



Data Visualisation
- Don't forget to present your final two data visualisation (one spatial and one non-spatial) and the justifications about the decisions you made whilst preparing and visualising the data.

## References
<p><a href="add_url_here">[1]</a>Footnote citation goes here</p>
<p><a href="https://github.com/FrancescaPontin/GEOG5990">[2]</a>GEOG5990M course materials</p>
<p><a href="add_url_here">[3]</a>Citation 3</p>